In [1]:
#Установка необходимых библиотек
!python -m venv ./venv
!source ./venv/bin/activate

!pip install TeleBot
!pip install selenium
!pip install aiohttp

"source" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


  Using cached telebot-0.0.5-py3-none-any.whl (4.8 kB)
  Using cached pytelegrambotapi-4.19.1-py3-none-any.whl (244 kB)
  Using cached selenium-4.21.0-py3-none-any.whl (9.5 MB)
  Using cached trio-0.25.1-py3-none-any.whl (467 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting un

In [2]:
#Подключение библиотек
import asyncio

from telebot import TeleBot
from telebot.types import Message

import asyncio

import asyncio

from selenium import webdriver
from selenium.webdriver.common.by import By

import asyncio

from telebot.async_telebot import AsyncTeleBot

In [3]:
# Telegram-bot token
TOKEN = '7212313830:AAFSskD7wxhzO62GGTnknkfPMJLnYPMLr7c'

# Scrapper settings
SCROLL_DELAY = 0.5
PAGE_LOAD_DELAY = 0.1

In [4]:
async def worker(query: str, art: str, message: Message, bot: TeleBot):
    result = await get_good_position(query, art)
    if result[0] != -1:
        page, position, absolute_position = result
        answer = (f'Артикул: {art}\n'
                  f'Страница: {page}, Позиция: {position}\n'
                  f'Абсолютная позиция: {absolute_position}\n'
                  f'Ссылка: {prepare_url(query, page)}')
        await bot.reply_to(message, answer)
        return

    await bot.reply_to(message, f'Артикул {art} не найден по данному запросу.')


async def find_item_position(message: Message, bot: TeleBot):
    if len(message.text.split()) < 2:
        await bot.reply_to(message, 'Не верный запрос.')
        return

    *query, art = message.text.split()
    query = '%20'.join(query)

    await bot.reply_to(message, 'В обработке...')
    asyncio.create_task(worker(query, art, message, bot))
from telebot import TeleBot
from telebot.types import Message


async def start_handler(message: Message, bot: TeleBot):
    text = (
        "Привет. Я могу найти позицию в поисковой выдаче WB по артикулу.\n"
        "Просто введи запрос и артикул.\n"
        "Например: \n"
        "    велосипед детский 145296859"
    )

#функция формирует URL для запроса на сайт Wildberries с учетом переданного запроса и номера страницы
    await bot.send_message(message.chat.id, text)
def prepare_url(query: str, page: int):
    return ('https://www.wildberries.ru/catalog/0/search.aspx'
            f'?page={page}&sort=popular&search={query}')


#функция создает экземпляр веб-драйвера Chrome с определенными настройками: скрытый режим, отключение загрузки изображений
def get_webdriver():
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument('--headless=new')
    chromeOptions.add_experimental_option('prefs',
                                          {'profile.managed_default_content_settings.images': 2})

    return webdriver.Chrome(options=chromeOptions)


""""
функция осуществляет поиск позиции товара с заданным артикулом на сайте Wildberries. 
Она использует веб-драйвер для загрузки страниц и скроллинга, чтобы найти нужный товар на каждой странице.
""""
async def get_good_position(query: str, art: str, start_page: int = 1) -> tuple[int]:

    with get_webdriver() as driver:
        driver.set_window_size(1680, 1000)

        page = start_page
        absolute_position = 0

        while True:
            # Подготавливаем URL-адрес и выполняем запрос.
            url = prepare_url(query, page)
            driver.get(url)

            # Ожидание загрузки страницы.
            # Индикаторами являются классы "разбивка на страницы" и "поиск не найден".
            while (not driver.find_elements(By.CLASS_NAME, 'pagination')
                   and len(driver.find_elements(By.CLASS_NAME, 'not-found-search')) == 0):
                await asyncio.sleep(PAGE_LOAD_DELAY)

            # Прекраoщаем поиск, если ничего не найдено
            if len(driver.find_elements(By.CLASS_NAME, 'not-found-search')) != 0:
                return (-1, -1, -1)

            # Прокрутка во время загрузки
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                # Скролим до последней карточки товара
                driver.execute_script("arguments[0].scrollIntoView(true);",
                                      driver.find_elements(
                                          By.XPATH,
                                          "//div[@class='product-card-list']/article")[-1])

                await asyncio.sleep(SCROLL_DELAY)

                
                new_height = driver.execute_script(
                    "return document.body.scrollHeight")
                if new_height == last_height:
                    break

                last_height = new_height

            elements = driver.find_elements(
                By.XPATH, "//div[@class='product-card-list']/article")

            # Поиск позиции нужной карточки
            for i, element in enumerate(elements):
                if element.get_dom_attribute('data-nm-id') == art:
                    return (page, i + 1, absolute_position + i + 1)

            absolute_position += len(elements)
            page += 1

In [5]:
#функция, которая регистрирует обработчики сообщений для бота
def register_handlers(bot: AsyncTeleBot):
    bot.register_message_handler(start_handler, commands=[
                                 'start'], pass_bot=True)
    bot.register_message_handler(
        find_item_position, content_types=['text'], pass_bot=True)


async def run():
    bot = AsyncTeleBot(TOKEN)
    register_handlers(bot)
    await bot.polling(non_stop=True)


await run()